In [2]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyodbc

## SQL Server Connection

In [2]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

### Our main issue is to create a connect using the pyodbc method. Whether you want to write what you want inside it or create it outside and write it below. The function here is actually to check whether the connection was successful or not.

In [3]:
driver = '{ODBC Driver 17 for SQL Server}'
server = '.\MSSQLSERVER2' # Current Local server 
database = 'master'#you can add existing DB you would like to work on
user = 'sa'
password = '---------'
conn = create_server_connection(driver, server, database, user, password)

Successfull


## Windows Authentication

#### What we do here is to create a trusted connection.

In [99]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=.\MSSQLSERVER2; database=master; TRUSTED_CONNECTION=yes;"

In [100]:
conn = pyodbc.connect(conn_string);  # connection objesi

In [101]:
crs = conn.cursor()  # database içindeki verilere satır satır ulaşmayı sağlayan araçtır

In [102]:
conn.autocommit = True # conn.commit() we did automatically

In [10]:
create_database_query = 'Create database test_1'

In [11]:
def create_database (conn, create_database_query):
    crs = conn.cursor()
    try:
        crs.execute(create_database_query)
        print("Database is created succesfully")
    except Error as err:
        print(f"Error:'{err}'")

In [12]:
create_database(conn, create_database_query)

Database is created succesfully


#### Another function to run query 

In [76]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [77]:
query = 'USE test_1'
execute_query(conn,query)

Query Succeessful!


## Create Table

#### We connected with Windows Authentication to avoid entering a password

In [16]:
query = 'CREATE TABLE Test_Table (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'
execute_query(conn, query)

Query Succeessful!


#### I have created Test_Table table under test1 database.

## Insert (table)

In [18]:
crs = conn.cursor()
crs.execute("INSERT Test_Table(FirstName,LastName) VALUES('Bob', 'Marley')")
#conn.commit()

***Bob Marley inserted***

In [34]:
crs.execute("INSERT Test_Table (FirstName, LastName) VALUES ('Bruce', 'Lee'),('Bart','Simpson')")

#### You can also insert with parameter like below

In [36]:
crs.execute("INSERT Test_Table (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')
crs.execute("INSERT Test_Table (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT Test_Table (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT Test_Table (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT Test_Table (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

## Delete

In [38]:
crs.execute("DELETE FROM Test_Table WHERE FirstName = 'Tom'")  # Deleted the id which is Tom

In [40]:
crs.execute("DELETE FROM Test_Table WHERE FirstName = ?" ,'Stefan')

## Rowcount 

#### Indicates the number of processes affected as a result of the last executed operation.

In [33]:
crs.rowcount

1

In [41]:
print(crs.rowcount,"row(s) deleted")

1 row(s) deleted


## Read Data

## fetchone()

#### It serves to capture and display the data in SQL without going to the database.

In [43]:
crs.execute('SELECT FirstName, LastName FROM Test_Table')
row = crs.fetchone() # tablodaki ilk kaydı getirir
row

('Bob', 'Marley')

In [44]:
row = crs.fetchone() # now skips the first data and returns the second record
row

('Bruce', 'Lee')

In [45]:
crs.execute('SELECT FirstName, LastName FROM Test_Table')
row = crs.fetchone() 
row

('Bob', 'Marley')

If you want it to start from the beginning, not to skip and go from the second one, it is necessary to write this again at the beginning.

***crs.execute('SELECT FirstName, LastName FROM TestA')***

In [46]:
row.FirstName  # returned the desired column data in the row where the cursor is present

'Bob'

In [47]:
row.LastName  

'Marley'

## fetchall()

In [49]:
crs.execute('SELECT ID, FirstName, LastName FROM Test_Table')
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (4, 'Bruce', 'Lee'),
 (5, 'Bart', 'Simpson'),
 (9, 'Jerry', 'Mouse'),
 (10, 'Owen', 'William')]

In [50]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

1 Bob Marley
2 Bruce Lee
4 Bruce Lee
5 Bart Simpson
9 Jerry Mouse
10 Owen William


In [52]:
crs.execute("""SELECT ID, FirstName, LastName
                FROM Test_Table""")
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (4, 'Bruce', 'Lee'),
 (5, 'Bart', 'Simpson'),
 (9, 'Jerry', 'Mouse'),
 (10, 'Owen', 'William')]

In [53]:
type(rows)  # datatype

list

In [55]:
# with filter

crs.execute("""SELECT ID, FirstName, LastName
               FROM Test_Table
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = crs.fetchall()
rows

[(10, 'Owen', 'William')]

## fetchmany()

Returns the record how many want

In [57]:
crs.execute("""SELECT ID, FirstName, LastName
               FROM Test_Table""")
rows = crs.fetchmany(5)
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (4, 'Bruce', 'Lee'),
 (5, 'Bart', 'Simpson'),
 (9, 'Jerry', 'Mouse')]

## Fetchval()

Bring the value

In [59]:
crs.execute("SELECT MAX(ID) FROM Test_Table").fetchval()

10

In [60]:
crs.execute("SELECT LastName FROM Test_Table").fetchval()

'Marley'

## Pandas Methods

In [61]:
import pandas as pd

#### Manually we tranformed data into Dataframe

In [78]:
def read_query (conn, query): #connection nesnesi yukarıda tanımlanmıştı.
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [79]:
query = """SELECT ID, FirstName, LastName FROM dbo.Test_Table"""
               

In [80]:
crs = conn.cursor()

In [81]:
result=read_query (conn, query)

In [82]:
from_db = []
for result in result:
    result = list(result)
    from_db.append(result)

In [83]:
columns = ['ID','FirstName', 'LastName']
df = pd.DataFrame(from_db, columns = columns)
df

,ID,FirstName,LastName
0,1,Bob,Marley
1,2,Bruce,Lee
2,4,Bruce,Lee
3,5,Bart,Simpson
4,9,Jerry,Mouse
5,10,Owen,William


#### With read_aql method easily converted data to Dataframe

In [103]:
query = 'use RFM'
execute_query(conn,query)

Query Succeessful!


In [95]:
crs.execute("SELECT * FROM Year2009").fetchmany(4)

[(510866.0, '21201', 'TROPICAL  HONEYCOMB PAPER GARLAND ', 1.0, datetime.datetime(2010, 6, 4, 9, 52), 2.55, None, 'United Kingdom'),
 (510866.0, '21206', 'STRAWBERRY HONEYCOMB  GARLAND ', 1.0, datetime.datetime(2010, 6, 4, 9, 52), 1.65, None, 'United Kingdom'),
 (510866.0, '84705D', 'CHERRY BLOSSOM PHOTO FRAME ', 1.0, datetime.datetime(2010, 6, 4, 9, 52), 4.65, None, 'United Kingdom'),
 (510866.0, '21214', 'SET OF 72 IVORY PAPER DOILIES', 1.0, datetime.datetime(2010, 6, 4, 9, 52), 1.45, None, 'United Kingdom')]

In [105]:
pd.read_sql("SELECT * FROM Year2009", con = conn) 

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,510866.0,21201,TROPICAL HONEYCOMB PAPER GARLAND,1.0,2010-06-04 09:52:00,2.55,NaN,United Kingdom
1,510866.0,21206,STRAWBERRY HONEYCOMB GARLAND,1.0,2010-06-04 09:52:00,1.65,NaN,United Kingdom
2,510866.0,84705D,CHERRY BLOSSOM PHOTO FRAME,1.0,2010-06-04 09:52:00,4.65,NaN,United Kingdom
3,510866.0,21214,SET OF 72 IVORY PAPER DOILIES,1.0,2010-06-04 09:52:00,1.45,NaN,United Kingdom
4,510866.0,84985A,SET OF 72 GREEN PAPER DOILIES,1.0,2010-06-04 09:52:00,1.45,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
525456,510866.0,47570B,TEA TIME TABLE CLOTH,1.0,2010-06-04 09:52:00,10.65,NaN,United Kingdom
525457,510866.0,22223,CAKE PLATE LOVEBIRD PINK,1.0,2010-06-04 09:52:00,4.95,NaN,United Kingdom
525458,510866.0,21199,PINK HEART CONFETTI IN TUBE,1.0,2010-06-04 09:52:00,1.65,NaN,United Kingdom
525459,510866.0,37506,NEW ENGLAND MILK JUG W GIFT BOX,1.0,2010-06-04 09:52:00,5.95,NaN,United Kingdom


In [104]:
df_prod = pd.read_sql("SELECT * FROM Year2009", con = conn)
df_prod.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,510866.0,21201,TROPICAL HONEYCOMB PAPER GARLAND,1.0,2010-06-04 09:52:00,2.55,NaN,United Kingdom
1,510866.0,21206,STRAWBERRY HONEYCOMB GARLAND,1.0,2010-06-04 09:52:00,1.65,NaN,United Kingdom
2,510866.0,84705D,CHERRY BLOSSOM PHOTO FRAME,1.0,2010-06-04 09:52:00,4.65,NaN,United Kingdom
3,510866.0,21214,SET OF 72 IVORY PAPER DOILIES,1.0,2010-06-04 09:52:00,1.45,NaN,United Kingdom
4,510866.0,84985A,SET OF 72 GREEN PAPER DOILIES,1.0,2010-06-04 09:52:00,1.45,NaN,United Kingdom
5,510866.0,21973,SET OF 36 MUSHROOM PAPER DOILIES,1.0,2010-06-04 09:52:00,1.45,NaN,United Kingdom
6,510866.0,21201,TROPICAL HONEYCOMB PAPER GARLAND,2.0,2010-06-04 09:52:00,2.55,NaN,United Kingdom
7,510866.0,21190,PINK HEARTS PAPER GARLAND,2.0,2010-06-04 09:52:00,1.65,NaN,United Kingdom
8,510866.0,84859A,SILVER DISCO HANDBAG,1.0,2010-06-04 09:52:00,5.95,NaN,United Kingdom
9,510866.0,22175,PINK OWL SOFT TOY,1.0,2010-06-04 09:52:00,2.95,NaN,United Kingdom


## DF to Database

#### You can find below the method after manipulating the data in pandas how to processed in SQL

In [106]:
import sqlalchemy
import urllib

In [107]:
create_database_query = 'Create database Tayfun'

In [108]:
def create_database (conn, create_database_query):
    crs = conn.cursor()
    try:
        crs.execute(create_database_query)
        print("Database is created succesfully")
    except Error as err:
        print(f"Error:'{err}'")

In [109]:
create_database(conn, create_database_query)

Database is created succesfully


In [113]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [122]:
crs.execute('USE RFM')

In [117]:
driver = '{ODBC Driver 17 for SQL Server}'
server = '.\MSSQLSERVER2' #localhost
database = 'RFM'#
user = 'sa'
password = 'Sakarya.1910'


In [118]:
params = 'DRIVER='+driver + ';SERVER=' + server +';PORT=1433;DATABASE='+ database + ';UID=' + user + ';PWD=' + password

In [119]:
db_params = urllib.parse.quote_plus (params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [123]:
df_prod = pd.read_sql("SELECT * FROM Year2009 where price >5.00", con = conn)
df_prod.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,510866.0,84859A,SILVER DISCO HANDBAG,1.0,2010-06-04 09:52:00,5.95,NaN,United Kingdom
1,510866.0,22191,EAU DE NIL DINER WALL CLOCK,1.0,2010-06-04 09:52:00,8.50,NaN,United Kingdom
2,510866.0,84923,PINK BUTTERFLY HANDBAG W BOBBLES,1.0,2010-06-04 09:52:00,8.50,NaN,United Kingdom
3,510866.0,21839,MUMMY MOUSE RED GINGHAM RIBBON,1.0,2010-06-04 09:52:00,5.45,NaN,United Kingdom
4,NaN,21527,RED RETROSPOT TRADITIONAL TEAPOT,-1.0,2010-06-04 09:53:00,7.95,12809.0,Portugal
5,NaN,22220,CAKE STAND LOVEBIRD 2 TIER WHITE,-3.0,2010-06-04 09:59:00,8.50,12424.0,Australia
6,510936.0,47590B,PINK HAPPY BIRTHDAY BUNTING,6.0,2010-06-04 10:35:00,5.45,12701.0,Germany
7,510936.0,POST,POSTAGE,2.0,2010-06-04 10:35:00,18.00,12701.0,Germany
8,510942.0,21755,LOVE BUILDING BLOCK WORD,3.0,2010-06-04 10:37:00,5.95,13715.0,United Kingdom
9,510942.0,22220,CAKE STAND LOVEBIRD 2 TIER WHITE,2.0,2010-06-04 10:37:00,9.95,13715.0,United Kingdom


In [124]:
df_prod.to_sql("price_greater_5", engine, index = False, if_exists = "append", schema = "dbo")

### When it comes to last column we create a table in SQL with the query which is SELECT * FROM Year2009 where price >5.00 We are able to observe the table in SQL  through to_sql method